In [1]:
from jupyter_innotater import Innotater
from jupyter_innotater.data import ImageInnotation, BoundingBoxInnotation, \
                                    MultiClassInnotation, BinaryClassInnotation

import numpy as np, os

### Image Filenames and Bounding Boxes

In [2]:
foodfns = sorted(os.listdir('./foods/'))
targets = np.zeros((len(foodfns), 4), dtype='int') # (x,y,w,h) for each data row

Innotater( ImageInnotation(foodfns, path='./foods'), BoundingBoxInnotation(targets) )

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [3]:
targets

array([[206, 103,  59,  54],
       [106, 125, 180, 128],
       [165,  64,  92,  84],
       [188, 110,  74,  70],
       [164,  39, 147, 147],
       [137, 130,  82,  76],
       [ 91,  96, 119, 119],
       [221,  35, 102, 120]])

In [4]:
# Write our newly-input bounding box data to disk - will be lost otherwise
import pandas as pd
df = pd.DataFrame(targets, columns=['x','y','w','h'])
df.insert(0,'filename', foodfns)
df.to_csv('./bounding_boxes.csv')

In [5]:
df

,filename,x,y,w,h
0,avocado.jpg,206,103,59,54
1,banana.jpg,106,125,180,128
2,garlic.jpg,165,64,92,84
3,gingerbiscuit.jpg,188,110,74,70
4,grapefruit.jpg,164,39,147,147
5,lime.jpg,137,130,82,76
6,onion.jpg,91,96,119,119
7,sweetpotato.jpg,221,35,102,120


### Numpy Image Data and Multi-classification

In [6]:
import cv2
classes = ['vegetable', 'biscuit', 'fruit']
foods = [cv2.imread('./foods/'+f) for f in foodfns]
targets = [0] * len(foodfns)

In [7]:
w2 = Innotater(
        ImageInnotation(foods, name='Food'), 
        MultiClassInnotation(targets, name='FoodType', classes=classes, desc='Food Type')
)
display(w2)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…

In [8]:
targets

[2, 2, 0, 1, 2, 2, 0, 0]

In [9]:
# Convert targets from a 1-dim array to one-hot representation - Innotater works with that just as well
onehot_targets = np.squeeze(np.eye(np.array(targets).max()+1)[np.array(targets).reshape(-1)]); onehot_targets

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [10]:
Innotater(
    ImageInnotation(foods, name='Food'), 
    MultiClassInnotation(onehot_targets, name='FoodType', classes=classes, desc='Food Type')
)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…

### Filenames and binary classification

Set image to display at a smaller width to make it more manageable - but bounding box co-ordinates would be relative to the unzoomed image.

In [12]:
isfruit_targets = (np.array(targets) == 2).astype('int')
w3 = Innotater( ImageInnotation(foodfns, path='./foods', width=300),
                BinaryClassInnotation(isfruit_targets, name='Is Fruit')
              )
display(w3)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [14]:
isfruit_targets

array([1, 1, 0, 0, 1, 1, 0, 0])

### Image Filenames and Binary Classification plus Bounding Boxes

Use indexes attribute to limit display just to the fruits where we want to add bounding boxes. Drop the indexes property if you also want to be able to check non-fruits.

In [16]:
bboxes = np.zeros((len(foodfns),4), dtype='int')
isfruits = np.expand_dims(isfruit_targets, axis=-1)

suspected_fruits = isfruits == 1 # Or you can specify an array/list of int indices

w6 = Innotater(
        ImageInnotation(foodfns, name='Food', path='./foods'), 
        [ BinaryClassInnotation(isfruits, name='Is Fruit'),
          BoundingBoxInnotation(bboxes, name='bbs', source='Food', desc='Food Type') ],
    indexes = suspected_fruits
)

display(w6)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [17]:
result = np.concatenate([isfruits,bboxes], axis=-1); result

array([[  1, 205, 101,  59,  59],
       [  1, 114, 129, 174, 128],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [  1, 167,  45, 148, 146],
       [  1, 136, 138,  86,  69],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0]])

### Image versus Image and Binary Classification

In [18]:
targets = np.array([[1,0]] * 5) # One-hot format, defaulting to 0 class
lfoods = foods[:5]
rfoods = lfoods.copy()
rfoods.reverse()

w5 = Innotater([ImageInnotation(lfoods, name='Food 1'), ImageInnotation(rfoods, name='Food 2')], 
        [BinaryClassInnotation(targets, name='Are Equal')])
display(w5)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…

In [19]:
targets

array([[1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0]])

### Two Image Data with named source of Bounding Boxes

In [20]:
foods2 = [f for f in foods]
targets = np.array([[i*10,i*20,i*30,i*40] for i in range(len(foods))])
targets2 = 2 * targets.copy() + 30

w4 = Innotater([ImageInnotation(foods, name='Food1'), 
                ImageInnotation(foods2, name='Food2')], 
        [
         BoundingBoxInnotation(targets, desc='Food Type1', source='Food1'), 
         BoundingBoxInnotation(targets2, desc='Food Type2', source='Food2')
        ])
display(w4)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…